# Building training dataset from OASST2

# Preparation

## Imports

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from src.convert_to_andalusian_spanish import AndalusianConverter
from transformers import AutoTokenizer
import plotly.express as px
import os

## Data

In [2]:
filename = os.path.join('data', 'raw', '2023-11-05_oasst2_all.messages.jsonl')
raw_df = pd.read_json(filename, lines=True)

# Filter for lang = 'es'
raw_df = raw_df[raw_df['lang'] == 'es']

# Drop unnecessary columns
to_drop = ['user_id', 'created_date', 'lang', 'emojis', 'model_name']
raw_df.drop(columns = to_drop, inplace = True)

print(raw_df.shape)
raw_df.head()

(47513, 14)


,message_id,parent_id,text,role,review_count,review_result,deleted,rank,synthetic,labels,events,detoxify,message_tree_id,tree_state
0,002c4715-b026-48d1-8d19-3f724a9fc1e8,None,Dame los pasos de las cosas que debería de apr...,prompter,3,1.0,False,NaN,False,"{'spam': {'value': 0.0, 'count': 4}, 'lang_mis...",NaN,"{'toxicity': 0.0023501887917510003, 'severe_to...",002c4715-b026-48d1-8d19-3f724a9fc1e8,ready_for_export
1,20004f60-337b-4962-b9d0-916be14509f9,002c4715-b026-48d1-8d19-3f724a9fc1e8,"Para ser un desarrollador de videojuegos, hay ...",assistant,3,1.0,False,0.0,False,"{'spam': {'value': 0.0, 'count': 4}, 'fails_ta...",NaN,"{'toxicity': 0.00023558732937000002, 'severe_t...",002c4715-b026-48d1-8d19-3f724a9fc1e8,ready_for_export
2,fe07c27a-e361-47b8-b935-4af6b9bd1836,002c4715-b026-48d1-8d19-3f724a9fc1e8,"Ante todo, debes entender que el desarrollo de...",assistant,3,1.0,False,1.0,False,"{'spam': {'value': 0.0, 'count': 5}, 'fails_ta...",NaN,"{'toxicity': 0.00021190838015100003, 'severe_t...",002c4715-b026-48d1-8d19-3f724a9fc1e8,ready_for_export
3,cd8019ba-3b37-4239-8bb0-25e9f2e08933,fe07c27a-e361-47b8-b935-4af6b9bd1836,"Vale, quiero asumir yo sólo todas las tareas. ...",prompter,3,1.0,False,NaN,False,"{'spam': {'value': 0.0, 'count': 3}, 'lang_mis...",NaN,"{'toxicity': 0.001551101100631, 'severe_toxici...",002c4715-b026-48d1-8d19-3f724a9fc1e8,ready_for_export
4,8254d75f-af1e-4d45-bad1-4db8dee773a8,cd8019ba-3b37-4239-8bb0-25e9f2e08933,Aquí tienes una lista de los pasos generales q...,assistant,3,1.0,False,0.0,False,"{'spam': {'value': 0.0, 'count': 3}, 'fails_ta...",NaN,"{'toxicity': 0.000269432144705, 'severe_toxici...",002c4715-b026-48d1-8d19-3f724a9fc1e8,ready_for_export


# Design working dataframe

## General format

In [3]:
# Create base copy
filtered_df = raw_df.copy()

# Drop rows where review_result is 1
filtered_df = filtered_df[filtered_df['review_result'] == 1]

# Keep needed columns
to_keep = ['message_id', 'text', 'role', 'message_tree_id', 'parent_id']
filtered_df = filtered_df[to_keep]

# Convert IDs to integers
id_dict = {k: v for v, k in enumerate(filtered_df['message_id'].unique(), start=1)}
to_map = ['message_id', 'message_tree_id', 'parent_id']
for col in to_map:
    filtered_df[col] = filtered_df[col].map(id_dict)

# Fill NaNs with 0
to_fill = ['message_tree_id', 'parent_id']
filtered_df[to_fill] = filtered_df[to_fill].fillna(0).astype(int)

# Set message_id as index
filtered_df.set_index('message_id', inplace=True)

print(filtered_df.shape)
filtered_df.head()

(46427, 4)


,text,role,message_tree_id,parent_id
message_id,,,,
1,Dame los pasos de las cosas que debería de apr...,prompter,1,0
2,"Para ser un desarrollador de videojuegos, hay ...",assistant,1,1
3,"Ante todo, debes entender que el desarrollo de...",assistant,1,1
4,"Vale, quiero asumir yo sólo todas las tareas. ...",prompter,1,3
5,Aquí tienes una lista de los pasos generales q...,assistant,1,4


In [4]:
# Save checkpoint
checkpoint = {'filtered_df': filtered_df}

## Obtain 2-element conversations

In [5]:
# Load checkpoint
temp_df = checkpoint['filtered_df']

In [6]:
# For each row in aux_df, create a column with the IDs of the messages in the same thread
def get_path(row: pd.Series,
             df: pd.DataFrame = temp_df) -> list:
    """
    Generate a list of message IDs that are in the same thread as the current message.

    Args:
    row (pd.Series): A row from a DataFrame.

    Returns:
    list: A list of message IDs.
    """

    path = [row.name]
    parent_id = row['parent_id']
    while parent_id != 0:
        path.append(parent_id)
        parent_id = df.loc[parent_id, 'parent_id']

    # Reverse the list
    path = path[::-1]

    return path

temp_df['path'] = temp_df.apply(get_path, axis=1)
display(temp_df)

,text,role,message_tree_id,parent_id,path
message_id,,,,,
1,Dame los pasos de las cosas que debería de apr...,prompter,1,0,[1]
2,"Para ser un desarrollador de videojuegos, hay ...",assistant,1,1,"[1, 2]"
3,"Ante todo, debes entender que el desarrollo de...",assistant,1,1,"[1, 3]"
4,"Vale, quiero asumir yo sólo todas las tareas. ...",prompter,1,3,"[1, 3, 4]"
5,Aquí tienes una lista de los pasos generales q...,assistant,1,4,"[1, 3, 4, 5]"
...,...,...,...,...,...
46423,Como se implementar un CRUD con front-end desa...,prompter,46423,0,[46423]
46424,¿Cuantos idiomas hay en el mundo? ¿Cuál se con...,prompter,46424,0,[46424]
46425,Cómo funcionan los emuladores de consolas en l...,prompter,46425,0,[46425]


In [7]:
# Extract paths with 2 elements
mask = temp_df['path'].apply(len) == 2
two_element_paths = temp_df[mask]['path'].tolist()
print('Two-element paths:', len(two_element_paths))

Two-element paths: 13020


In [8]:
def build_conversation_dict(path: list,
                       df: pd.DataFrame = temp_df) -> pd.DataFrame:
    """
    Build a conversation from a list of message IDs as a JSON object.

    Args:
    path (list): A list of message IDs.
    df (pd.DataFrame): A DataFrame with the messages.

    Returns:
    pd.DataFrame: A DataFrame with the conversation.
    """

    conversation_dict = {
        'input': df.loc[path[0], 'text'],
        'output': df.loc[path[1], 'text'],
        'path': path,
    }

    return conversation_dict

conversation_dict_list = [build_conversation_dict(path) for path in two_element_paths]
conversation_df = pd.DataFrame(conversation_dict_list)
print(conversation_df.shape)
conversation_df.head()

(13020, 3)


,input,output,path
0,Dame los pasos de las cosas que debería de apr...,"Para ser un desarrollador de videojuegos, hay ...","[1, 2]"
1,Dame los pasos de las cosas que debería de apr...,"Ante todo, debes entender que el desarrollo de...","[1, 3]"
2,Dame los pasos de las cosas que debería de apr...,No hay un plan perfecto para convertirte en un...,"[1, 9]"
3,Hoy me apetece comer un plato que tenga pescad...,"Claro, aquí van algunas opciones para un plato...","[27, 28]"
4,Hoy me apetece comer un plato que tenga pescad...,"Claro, con gusto.\nPrimero asumiré que estás d...","[27, 30]"


In [9]:
# Save checkpoint
checkpoint['conversation_df'] = conversation_df

### Apply Andalusian transliteration

In [10]:
# Load checkpoint
conversation_df = checkpoint['conversation_df']

In [11]:
# Convert the text to Andalusian Spanish
converter = AndalusianConverter()
conversation_df['input'] = conversation_df['input'].apply(converter.convert)
conversation_df['output'] = conversation_df['output'].apply(converter.convert)
print(f'Final shape: {conversation_df.shape}')

Final shape: (13020, 3)


In [12]:
# Test converter
converter.convert('Paula tiene un perro llamado Mushu, ¿a que todos amamos a Mushu? ')

'Paula ʌiene un perro yamáo Muъu, ¿a ce ʌoь amamoь a Muъu?'

In [13]:
print(f'Final shape: {conversation_df.shape}')

Final shape: (13020, 3)


In [14]:
# Show results
pd.set_option('display.max_colwidth', None)
conversation_df.sample(5)

,input,output,path
12784,"¿Cé ʌan imporʌanʌe eь el idioma ""Eьpañó"" (aбlando inʌennaɿionalmenʌe), en funɿión, del deъempeño de la comunicaɿión del miъmo, con perъonaь d'oʌroь paíъeь; i ъu fleьъiбilidá como idioma, en comparaɿión con oʌroь idiomaь eьʌranьeroь? (pa la implemenʌaɿión como idioma preferío por perъonaь ce no aбlan eьpañó naʌiбo).","El idioma eьpañó eь uno de loь idiomaь máь aбláoь en el mundo, con máь de 580 miyoneь d'aбlanʌeь en ʌo'l mundo, lo ce lo comбierʌe'n el ъeƨundo idioma máь aбláo deьpuéь del чino mandarín. Ademáь, el eьpañó eь el ʌerɿé idioma máь uʌiliɿáo'n inʌenné deьpuéь del inƨléь i el чino mandarín.\n\nEl eьpañó eь un idioma mu fleьъiбle i adaьʌaбle, lo ce lo aɿe fáɿilmenʌe aьɿeъiбle pa aceyoь ce deъen aprendello como ъeƨundo idioma. Ademáь, el eьpañó ъ'a comбerʌío'n un idioma imporʌanʌe'n el mundo de loь neƨoɿioь i la políʌica, lo ce lo aɿe mu uʌi pa la comunicaɿión inʌennaɿioná.\n\nLa fleьъiбilidá i la imporʌanɿia del eьpañó'n el conʌeьʌo inʌennaɿioná aɿen ce ъea una oьɿión aʌraьʌiбa pa aceyoь ce бuьcan aprendé un ъeƨundo idioma. Ademáь, el eчo de c'el eьpañó ъea aбláo'n muчoь paíъeь diferenʌeь ъiьnifica c'ai una ƨran canʌidá de recurъoь diьponiбleь pa aprendé'l idioma, como curъoь en línea, aplicaɿioneь i liбroь de ʌeьʌo.\n\nEn reъumen, el eьpañó eь un idioma imporʌanʌe i mu uʌi en el conʌeьʌo inʌennaɿioná deбío a ъu fleьъiбilidá, ъu adaьʌaбilidá i ъu amplia difuъión en el mundo. Eьʌo lo comбierʌe'n una oьɿión aʌraьʌiбa pa aceyoь ce бuьcan aprendé un ъeƨundo idioma pa meьorá ъu capaɿidá de comunicaɿión i ъu deъempeño'n conʌeьʌoь inʌennaɿionaleь.","[45643, 45646]"
11086,"Neɿeъiʌo ce ъe m'ofreьca una ƨía de cómo eбaluá la calidá d'una noʌiɿia leía por inʌenné, como una ъerie de paъoь a ъeƨí i criʌerioь lo máь oььeʌiбoь poъiбleь en un número ъufiɿienʌe pero no eьɿeъiбo ni compleьo.","Ací ʌ'ofreьco una ƨía con alƨunoь paъoь i criʌerioь ce pueden ayudarʌe a eбaluá la calidá d'una noʌiɿia leía por inʌenné:\n\nБerifica la fuenʌe: Aъeƨúraʌe de ce la fuenʌe de la noʌiɿia eь confiaбle. Бuьca informaɿión ъoбre'l ъiʌio wé o la orƨaniɿaɿión ce puбlicó la noʌiɿia. Ъi eь un ъiʌio wé deьconoɿí'o poco conoɿío, eь poъiбle ce deъeь imбeьʌiƨá máь anʌeь de confiá'n la informaɿión ce ъe preъenʌa.\n\nБuьca corroбoraɿión: Бerifica ce la noʌiɿia eь ɿierʌa conъulʌand'oʌraь fuenʌeь confiaбleь. Ъi no puedeь enconʌrá oʌra fuenʌe ce reьpalde la noʌiɿia, eь poъiбle ce deъeь dudá de ъu бeraɿidá.\n\nEьъamina'l ʌono: Eьъamina'l ʌono de la noʌiɿia. ¿Pareɿe imparɿiá o ъeьƨáo? Ъi pareɿe ъeьƨáo, eь poъiбle ce deъeь бuьcá oʌra fuenʌe c'ofreьca una perьpeьʌiбa máь eciliбrá.\n\nБerifica la feчa: Aъeƨúraʌe de ce la noʌiɿia no ъea anʌiwa o deъaьʌualiɿá. A menúo, laь noʌiɿiaь falъaь o enƨañoъaь ъe propaƨan en línea deьpuéь d'un eбenʌo reá, pero a menúo eьʌán má informáь o eьъaьeráь.\n\nБerifica loь eчoь: Бerifica ce loь eчoь preъenʌáoь en la noʌiɿia ъean preɿiъoь. Ъi ai afirmaɿioneь en la noʌiɿia ce ʌe pareɿen dudoъaь, eь poъiбle ce deъeь imбeьʌiƨallaь por ʌi miъmo anʌeь d'aɿeьʌallaь como бerdaderaь.\n\nEьъamina laь imáьeneь: Ъi ai imáьeneь en la noʌiɿia, aъeƨúraʌe de ce ъean auʌénʌicaь. A menúo, laь imáьeneь pueden ъé manipuláь o ъacáь de conʌeьʌo pa apoyá una narraʌiбa eьpeɿífica.\n\nImбeьʌiƨa al auʌó: Aбeriwa cién eьcriбió la noʌiɿia i c'eьperienɿia ʌienen en el ʌema. Ъi el auʌó eь deьconoɿí'o ʌiene una aьenda políʌica clara, eь poъiбle ce deъeь conъiderá la informaɿión con precauɿión.\n\nɅen en cuenʌa'l conʌeьʌo: Aъeƨúraʌe de conъiderá'l conʌeьʌo máь amplio'n el ce ъe preъenʌa la noʌiɿia. A бeɿeь, loь eбenʌoь pueden ъé preъenʌáoь de manera enƨañoъa o ъeьƨá pa apoyá una narraʌiбa eьpeɿífica.","[39779, 39781]"
9221,¿Creь c'el reɿienʌe auьe de laь inʌeliьenɿiaь arʌifiɿialeь afeьʌara de forma poъiʌiбa o neƨaʌiбa a la umanidá?,"Eь difiɿi ъaбello ya ce nadie puede predeɿí el fuʌuro, ъin emбarƨo ъi ъe noʌaran camбioь en el corʌo i larƨo plaɿo, ʌaleь como l'auʌomaʌiɿaɿión de ʌareaь repeʌiʌiбaь o la faɿiliʌaɿión de daʌoь al alcanɿe de ʌoь, ъi бi

In [15]:
# Save checkpoint
checkpoint['andalusian_transcript'] = conversation_df

### Trim maximum token length

In [16]:
# Load checkpoint
temp_df = checkpoint['andalusian_transcript']

In [17]:
# Load tokenizer
base_model_id = "mistralai/Mistral-7B-v0.1"

tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

c:\Users\jaime.garcia\OneDrive - Havas\Desktop\Proyectos\Personal\MistrAND-7B-v1\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


#### Input length distribution

In [18]:
input_lengths = temp_df['input'].apply(tokenizer.tokenize).apply(len)

# Plot input lengths
fig = px.histogram(input_lengths, title='Input Length Distribution')
fig.update_layout(title_x=0.5)
fig.show()

In [19]:
# Cumulative distribution
fig = px.histogram(input_lengths, 
                   cumulative=True, 
                   title='Input Length Cumulative Distribution', 
                   histnorm='probability',
                   labels = {'value': 'Input Length', 
                             'probability': 'Cumulative Probability',
                             'variable': 'Frequency'})
fig.update_layout(title_x=0.5)
fig.show()

In [20]:
# Compute percentage of inputs that are longer than n tokens
max_length = 250
mask = input_lengths > max_length
long_inputs = input_lengths[mask]
percentage = len(long_inputs) / len(input_lengths) * 100
print(f'Percentage of inputs longer than {max_length} tokens: {percentage:.2f}%')

Percentage of inputs longer than 250 tokens: 1.94%


#### Output length distribution

In [21]:
# Output length distribution
output_lengths = temp_df['output'].apply(tokenizer.tokenize).apply(len)

# Plot output lengths
fig = px.histogram(output_lengths, title='Output Length Distribution')
fig.update_layout(title_x=0.5)
fig.show()

In [22]:
# Cumulative distribution
fig = px.histogram(output_lengths, 
                   cumulative=True, 
                   title='Output Length Cumulative Distribution', 
                   histnorm='probability',
                   labels = {'value': 'Output Length', 
                             'probability': 'Cumulative Probability',
                             'variable': 'Frequency'})
fig.update_layout(title_x=0.5)
fig.show()

In [23]:
# Compute percentage of outputs that are longer than n tokens
max_length = 1600
mask = output_lengths > max_length
long_outputs = output_lengths[mask]
percentage = len(long_outputs) / len(output_lengths) * 100
print(f'Percentage of outputs longer than {max_length} tokens: {percentage:.2f}%')

Percentage of outputs longer than 1600 tokens: 1.96%


#### Combined distribution

In [24]:
# Set formatting function
def formatting_func(example):
    text = f"### Preƨunʌa: {example['input']}\n ### Γeьpueьʌa: {example['output']}"
    return text

In [25]:
# Apply formatting function
temp_df['formatted'] = temp_df.apply(formatting_func, axis=1)

In [26]:
# Combined length distribution
combined_lengths = temp_df['formatted'].apply(tokenizer.tokenize).apply(len)

# Plot combined lengths
fig = px.histogram(combined_lengths, title='Combined Length Distribution')
fig.update_layout(title_x=0.5)
fig.show()

In [27]:
# Cumulative distribution
fig = px.histogram(combined_lengths, 
                   cumulative=True, 
                   title='Combined Length Cumulative Distribution', 
                #    histnorm='probability',
                   labels = {'value': 'Combined Length', 
                             'probability': 'Cumulative Probability',
                             'variable': 'Frequency'})
fig.update_layout(title_x=0.5)
fig.show()

In [28]:
# Compute percentage of combined lengths that are longer than n tokens
max_length = 1250
mask = combined_lengths > max_length
long_combined = combined_lengths[mask]
percentage = len(long_combined) / len(combined_lengths) * 100
print(f'Percentage of combined lengths longer than {max_length} tokens: {percentage:.2f}% (n = {len(long_combined)})')

Percentage of combined lengths longer than 1250 tokens: 6.81% (n = 887)


In [29]:
# Compute percentage of combined lengths that are shorter than n tokens
max_length = 125
mask = combined_lengths < max_length
short_combined = combined_lengths[mask]
percentage = len(short_combined) / len(combined_lengths) * 100
print(f'Percentage of combined lengths shorter than {max_length} tokens: {percentage:.2f}% (n = {len(short_combined)})')

Percentage of combined lengths shorter than 125 tokens: 10.68% (n = 1391)


#### Trim to selected length

In [30]:
max_allowed_length = 1250
print(f'Number of examples before filtering: {len(temp_df)}')
temp_df = temp_df[combined_lengths <= max_allowed_length]
print(f'Number of examples after filtering: {len(temp_df)}')
temp_df.head()

Number of examples before filtering: 13020
Number of examples after filtering: 12133


,input,output,path,formatted
0,Dame loь paъoь de laь coъaь ce deбería d'aprendé pa ъé un deъarroyaó de бideoьueƨoь.,"Para ъé un deъarroyaó de бideoьueƨoь, ai una ъerie de paъoь ce ъe deбen ъeƨí.\n\n1. Aprendé un lenwaьe de proƨramaɿión. Loь lenwaьeь máь comuneь pa proƨramá бideoьueƨoь ъon C++, C# i Ьaбa.\n\n2. Aprendé un moʌó de ьueƨo. Loь moʌoreь de ьueƨo máь populareь ъon Uniʌy, Unreá Enьine i ƧameMacé Ъʌudío.\n\n3. Aprendé una erramienʌa de diъeño de ьueƨoь. Erramienʌaь como Coьʌrú 3 i ƧameЪalá ъon comuneь pa diъeñá ьueƨoь.\n\n4. Aprendé una erramienʌa de modeláo 3D. Erramienʌaь como Бlendé i Maya ъon populareь pa creá ƨráficoь 3D.\n\n5. Aprendé una erramienʌa d'animaɿión 3D. Erramienʌaь como Ⴈinema 4D i Maya ъon populareь pa creá animaɿioneь 3D.\n\n6. Deъarroyá aбilidadeь de diъeño de ьueƨoь. Eьʌo incluye'l diъeño de mecánicaь de ьueƨo, el diъeño de niбeleь, el diъeño de perъonaьeь i la proƨramaɿión de lóьica de ьueƨo.\n\n7. Aprendé una erramienʌa de ъonío. Erramienʌaь como Audaɿiʌy i Loьí Pro ъon populareь pa creá efeьʌoь de ъonío i múъica pa бideoьueƨoь.\n\n8. Aprendé una erramienʌa de diъeño ƨráfico. Erramienʌaь como Poʌoъó, Iyuьʌraʌó i ЬIMP ъon populareь pa creá ƨráficoь pa loь бideoьueƨoь.\n\n9. Aprendé a puбlicá un ьueƨo. Plaʌaformaь como Ъʌeán, iOЪ i Androí ʌienen ъuь propioь proɿeъoь de puбlicaɿión.\n\n10. Aprendé aбilidadeь de marceʌí. Eьʌo incluye'l uъo de redeь ъoɿialeь, la creaɿión de conʌenío i la ьeneraɿión de ʌráfico pa promoɿioná un ьueƨo.","[1, 2]","### Preƨunʌa: Dame loь paъoь de laь coъaь ce deбería d'aprendé pa ъé un deъarroyaó de бideoьueƨoь.\n ### Γeьpueьʌa: Para ъé un deъarroyaó de бideoьueƨoь, ai una ъerie de paъoь ce ъe deбen ъeƨí.\n\n1. Aprendé un lenwaьe de proƨramaɿión. Loь lenwaьeь máь comuneь pa proƨramá бideoьueƨoь ъon C++, C# i Ьaбa.\n\n2. Aprendé un moʌó de ьueƨo. Loь moʌoreь de ьueƨo máь populareь ъon Uniʌy, Unreá Enьine i ƧameMacé Ъʌudío.\n\n3. Aprendé una erramienʌa de diъeño de ьueƨoь. Erramienʌaь como Coьʌrú 3 i ƧameЪalá ъon comuneь pa diъeñá ьueƨoь.\n\n4. Aprendé una erramienʌa de modeláo 3D. Erramienʌaь como Бlendé i Maya ъon populareь pa creá ƨráficoь 3D.\n\n5. Aprendé una erramienʌa d'animaɿión 3D. Erramienʌaь como Ⴈinema 4D i Maya ъon populareь pa creá animaɿioneь 3D.\n\n6. Deъarroyá aбilidadeь de diъeño de ьueƨoь. Eьʌo incluye'l diъeño de mecánicaь de ьueƨo, el diъeño de niбeleь, el diъeño de perъonaьeь i la proƨramaɿión de lóьica de ьueƨo.\n\n7. Aprendé una erramienʌa de ъonío. Erramienʌaь como Audaɿiʌy i Loьí Pro ъon populareь pa creá efeьʌoь de ъonío i múъica pa бideoьueƨoь.\n\n8. Aprendé una erramienʌa de diъeño ƨráfico. Erramienʌaь como Poʌoъó, Iyuьʌraʌó i ЬIMP ъon populareь pa creá ƨráficoь pa loь бideoьueƨoь.\n\n9. Aprendé a puбlicá un ьueƨo. Plaʌaformaь como Ъʌeán, iOЪ i Androí ʌienen ъuь propioь proɿeъoь de puбlicaɿión.\n\n10. Aprendé aбilidadeь de marceʌí. Eьʌo incluye'l uъo de redeь ъoɿialeь, la creaɿión de conʌenío i la ьeneraɿión de ʌráfico pa promoɿioná un ьueƨo."
1,Dame loь paъoь de laь coъaь ce deбería d'aprendé pa ъé un deъarroyaó de бideoьueƨoь.,"Anʌe ʌo, deбeь enʌendé c'el deъarroyo de бideoьueƨoь eь una ʌarea compleьa ce reciere la parʌiɿipaɿión de diferenʌeь roleь: proƨramadoreь, diъeñadoreь ƨráficoь, ƨioniьʌaь, produьʌoreь, eьɿ. La primera deɿiъión ce deбeráь ʌomá poь, eь en cua de loь diferenʌeь roleь deъeaь eьpeɿialiɿarʌe o ъi preʌendeь aъumí ʌú ъolo ʌoaь laь ʌareaь.\nEn ьenerá, ъería бuena idea informarʌe, a ʌraбéь de бariaь fuenʌeь, de cuáleь ъon laь ʌareaь eъenɿialeь ce loь diьʌinʌoь roleь implican i de loь méʌodoь i luƨareь idóneoь pa formarʌe.\nA parʌí d'aí, aɿerʌe un plan de formaɿión, ʌeniendo'n cuenʌa ʌuь aьʌiʌudeь, ʌu luƨá de reъidenɿía i ʌuь finanɿaь.\nEьъiьʌen ɿenʌroь de formaɿión eьpeɿialiɿáoь, ʌanʌo ""on line"" como preъenɿialeь, ce ʌe ъerían de ƨran ayúa. O, ъi ʌe ъienʌeь capáь, puedeь aprendé por ʌu cuenʌa, uʌiliɿando, por eьemplo, la ƨran canʌidá de recurъoь c'ai diьponiбleь en Inʌenné.","[1, 3]","### Preƨunʌa: Dame loь paъoь de laь coъaь c

In [31]:
# Save checkpoint
checkpoint['trimmed_transcript'] = temp_df.copy()

### Save results

In [32]:
# Load checkpoint
full_df = checkpoint['trimmed_transcript']

In [33]:
# Retain only needed columns
to_keep = ['input', 'output']
full_df = full_df[to_keep].copy()

#### Full dataset

One of the following modes must be set:
* **preprod**: used for hyperparameter tuning.
* **prod**: used for final model fine-tuning.

It only affects the final file naming, but ensures that the naming conventions are consistent accross the repository.

In [ ]:
# Set saving mode
mode = 'prod' # Select either 'preprod' or 'prod'

In [ ]:
# Set saving settings
directory = os.path.join('data', 'processed')
filename = f'conversations_2E_ES_AND_{mode}_full.jsonl'
full_path = os.path.join(directory, filename)

In [ ]:
# Shuffle dataset
full_df = full_df.sample(frac=1, random_state=33)

In [ ]:
# Save as .jsonl
full_df.to_json(full_path, orient='records', lines=True)

#### Train and eval sets

If you plan to track the validation loss during training, it is recommended to set the number of evaluation instances to a fixed low number: since all samples are tested, keeping the usual 20 % of the data for evaluation results in longer training times.

In [ ]:
# Set saving settings
n_eval_instances = 1000

In [ ]:
# Split dataset
train_df = full_df[:-n_eval_instances]
val_df = full_df[-n_eval_instances:]
print(f'Training set shape: {train_df.shape}')
print(f'Validation set shape: {val_df.shape}')

In [ ]:
# Save training set as .jsonl
filename = f'conversations_2E_ES_AND_{mode}_train.jsonl'
filepath = os.path.join(directory, filename)
train_df.to_json(filename, orient='records', lines=True)

In [ ]:
# Save validation set as .jsonl
filename = f'conversations_2E_ES_AND_{mode}_val.jsonl'
filepath = os.path.join(directory, filename)
val_df.to_json(filename, orient='records', lines=True)

#### Upload to Hugging Face

In [ ]:
# Log into Hugging Face
# Alternatively, use !huggingface-cli login --token 
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# TODO: